In [1233]:
#pip install mido

In [1234]:
#pip install pygame

In [1235]:
#pip install fluidsynth

In [1236]:
# you also have to install the fluidynth exe using chocolatey
# see the fluidsynth and chocolatey website for details
# you also have to install the timidity exe from the timidity website

In [1237]:
import os
import numpy as np
import mido
import subprocess
import pygame
import random
from mido import Message, MidiFile, MidiTrack

In [1238]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

In [1239]:
cmajorscale = []
for n in range(72,84): # 0, 12, 24 etc are C. start at 72 to keep the melody in the higher register
    if n%12 in [0,2,4,5,7,9,11]:
        cmajorscale.append(n)

cmajchords = np.matrix([[48, 50, 52, 53, 55, 57, 59], # this matrix is used to build diatonic chord progressions in the key of C Maj
[52, 53, 55, 57, 59, 60, 62],
[55, 57, 59, 60, 62, 64, 65],
[59, 60, 62, 64, 65, 67, 69]])

print(cmajorscale)
print(cmajchords)
print(np.array(cmajorscale)+5)
print(cmajchords+5)

[72, 74, 76, 77, 79, 81, 83]
[[48 50 52 53 55 57 59]
 [52 53 55 57 59 60 62]
 [55 57 59 60 62 64 65]
 [59 60 62 64 65 67 69]]
[77 79 81 82 84 86 88]
[[53 55 57 58 60 62 64]
 [57 58 60 62 64 65 67]
 [60 62 64 65 67 69 70]
 [64 65 67 69 70 72 74]]


In [1240]:
# build the music generation algorithm here...

def generate_random_dicts(num_dicts, keys, weights, max_length):
    return [
        {k: random.randint(0, 6) for k in sorted(random.choices(keys, weights=weights, k=random.randint(1, max_length)))}
        for _ in range(num_dicts)
    ]

def volume_control(input_number):
    if not (0 <= input_number <= 6):
        raise ValueError("Input must be between 0 and 6 (inclusive).")
    # Calculate a base value based on the input number
    base_value = 50 + (input_number * 5)  # This gives a base value from 50 to 80
    # Introduce randomness: we can add a random value that can be negative or positive
    random_variation = np.random.randint(-5, 6)  # Random value between -5 and 5
    # Calculate the final value
    final_value = base_value + random_variation
    # Ensure the final value is within the range of 50 to 80
    return max(50, min(final_value, 80))

def weighted_random_choice(prev_value, max_value=6, weight_decay=2):
    """
    Generate a random value based on a weighted probability distribution
    that decreases as the distance from the previous value increases.
    """
    values = list(range(max_value + 1))
    weights = [1 / (1 + weight_decay * abs(value - prev_value)) for value in values]
    total_weight = sum(weights)
    normalized_weights = [w / total_weight for w in weights]
    return random.choices(values, weights=normalized_weights, k=1)[0]

def melody_smoother(data):
    """
    Update the values in a list of dictionaries based on a probability distribution
    related to the distance to the previous value.
    """
    updated_data = []
    prev_value = None  # Initialize with None for the first value
    for dictionary in data:
        updated_dict = {}
        for key, value in dictionary.items():
            if prev_value is None:
                # Assign the first value randomly
                updated_value = random.randint(0, 6)
            else:
                # Use the weighted random choice for subsequent values
                updated_value = weighted_random_choice(prev_value)
            updated_dict[key] = updated_value
            prev_value = updated_value 
        updated_data.append(updated_dict)
    return updated_data

def melody_declasher(chord_prog, melody_data):
    clashing_notes = {0:[3],2:[0,3],4:[0], 5:[3], 6:[0]}
    for chrd in range(len(melody_data)):
        for note in melody_data[chrd]:
            if note == num_subdiv and chrd < len(melody_data)-1:
                if chord_prog[chrd+1] in clashing_notes.keys() and melody_data[chrd][note] in clashing_notes[chord_prog[chrd+1]]:
                    if melody_data[chrd][note] == 0:
                        note_change = 1
                    else:
                        note_change = random.choice([-1,1])
                    melody_data[chrd][note] = melody_data[chrd][note] + note_change
            if note == num_subdiv and chrd == len(melody_data)-1:
                if chord_prog[0] in clashing_notes.keys() and melody_data[chrd][note] in clashing_notes[chord_prog[0]]:
                    if melody_data[chrd][note] == 0:
                        note_change = 1
                    else:
                        note_change = random.choice([-1,1])
                    melody_data[chrd][note] = melody_data[chrd][note] + note_change
            else: 
                if chord_prog[chrd] in clashing_notes.keys() and melody_data[chrd][note] in clashing_notes[chord_prog[chrd]]:
                    if melody_data[chrd][note] == 0:
                        note_change = 1
                    else:
                        note_change = random.choice([-1,1])
                    melody_data[chrd][note] = melody_data[chrd][note] + note_change
    return melody_data
    
def build_track(melody, bass, tenor, alto, lead):
    """
    all data, including the start and end of every single note, must be specified in sequential order.
    the purpose of this function is to automatically rearrange everything in this way 
    """
    cmajorscale = []
    for n in range(72,84): # 0, 12, 24 etc are C. start at 72 to keep the melody in the higher register
        if n%12 in [0,2,4,5,7,9,11]:
            cmajorscale.append(n)

    cmajchords = np.matrix([[48, 50, 52, 53, 55, 57, 59], # this matrix is used to build diatonic chord progressions in the key of C Maj
    [52, 53, 55, 57, 59, 60, 62],
    [55, 57, 59, 60, 62, 64, 65],
    [59, 60, 62, 64, 65, 67, 69]])
    
    voices = {'melody':melody, 'bass':bass, 'tenor':tenor, 'alto':alto, 'lead':lead}
    voice_chord_index = ['bass','tenor','alto','lead']
    note_status = {} # the status dict remembers the current note of each voice so each note can be ended
    volume_status = {}
    t=0 # "t" keeps track of the amount of time that has elapsed
    for chord in range(len(chord_progression)):
        for beat in range(num_subdiv):
            timegap = True # indicates whether or not for the very first time we are starting or ending a note at a new subsequent time. otherwise the "time" arg should be 0
            modulating = False 
            if beat == int(num_subdiv/2) and chord != 0: # when and how to modulate keys!
                if random.random() < 0.05:  # 5% chance
                    modulating = True
                    print('chord: ',chord_progression[chord],' - modulating')
                    modulation = np.random.choice([-5,-2,2,5],p=[.15,.15,.30,.40])
                    volume_status[voice] = volume_control(chord_progression[chord])-10
                    if timegap == True:
                        track.append(Message('note_off', note=cmajchords[0,chord_progression[chord]], velocity=volume_status['bass'], time=int(chord*measure_length+beat*measure_length/num_subdiv-t)))
                        t = int(chord*measure_length+beat*measure_length/num_subdiv)
                        timegap = False      
                        track.append(Message('note_off', note=cmajchords[1,chord_progression[chord]], velocity=volume_status['tenor'], time=0))
                        track.append(Message('note_off', note=cmajchords[2,chord_progression[chord]], velocity=volume_status['alto'], time=0))
                        track.append(Message('note_off', note=cmajchords[3,chord_progression[chord]], velocity=volume_status['lead'], time=0))
                        track.append(Message('note_off', note=cmajorscale[note_status['melody']], velocity=volume_status['melody'], time=0))
                        cmajorscale = np.array(cmajorscale) + modulation
                        cmajchords = cmajchords + modulation
                        track.append(Message('note_on', note=cmajchords[0,chord_progression[chord]], velocity=volume_status['bass'], time=0))
                        track.append(Message('note_on', note=cmajchords[1,chord_progression[chord]], velocity=volume_status['tenor'], time=0))
                        track.append(Message('note_on', note=cmajchords[2,chord_progression[chord]], velocity=volume_status['alto'], time=0))
                        track.append(Message('note_on', note=cmajchords[3,chord_progression[chord]], velocity=volume_status['lead'], time=0))
                        melody_reset = random.choice([1,2,4,6])
                        track.append(Message('note_on', note=cmajorscale[melody_reset], velocity=volume_status[voice], time=0))
                        note_status['bass'] = 0
                        note_status['tenor'] = 1
                        note_status['alto'] = 2
                        note_status['lead'] = 3
                        note_status['melody'] = melody_reset
                    else:
                        track.append(Message('note_off', note=cmajchords[0,chord_progression[chord]], velocity=volume_status['bass'], time=0))    
                        track.append(Message('note_off', note=cmajchords[1,chord_progression[chord]], velocity=volume_status['tenor'], time=0))
                        track.append(Message('note_off', note=cmajchords[2,chord_progression[chord]], velocity=volume_status['alto'], time=0))
                        track.append(Message('note_off', note=cmajchords[3,chord_progression[chord]], velocity=volume_status['lead'], time=0))
                        cmajorscale = np.array(cmajorscale) + modulation
                        cmajchords = cmajchords + modulation
                        track.append(Message('note_on', note=cmajchords[0,chord_progression[chord]], velocity=volume_status['bass'], time=0))
                        track.append(Message('note_on', note=cmajchords[1,chord_progression[chord]], velocity=volume_status['tenor'], time=0))
                        track.append(Message('note_on', note=cmajchords[2,chord_progression[chord]], velocity=volume_status['alto'], time=0))
                        track.append(Message('note_on', note=cmajchords[3,chord_progression[chord]], velocity=volume_status['lead'], time=0))
                        melody_reset = random.choice([1,2,4,6])
                        track.append(Message('note_on', note=cmajorscale[melody_reset], velocity=volume_status[voice], time=0))
                        note_status['bass'] = 0
                        note_status['tenor'] = 1
                        note_status['alto'] = 2
                        note_status['lead'] = 3 
                        note_status['melody'] = melody_reset
            for voice in voices:
                if beat in voices[voice][chord] and modulating == False:
                    if chord == 0 and beat == min(voices[voice][0]): 
                        pass # we don't need to end a previous note if it is the first note
                    else: # specify to end any notes that are already playing ("note_off" messages) before new notes can be started ("note_on" messages).
                        if voice == 'melody': # the melody is sourced from the scale vector, not the chord matrix
                            if timegap == True: 
                                track.append(Message('note_off', note=cmajorscale[note_status[voice]], velocity=volume_status[voice], time=int(chord*measure_length+beat*measure_length/num_subdiv-t)))
                                t = int(chord*measure_length+beat*measure_length/num_subdiv)
                                timegap = False
                            else: 
                                track.append(Message('note_off', note=cmajorscale[note_status[voice]], velocity=volume_status[voice], time=0))
                        else: # the voices other than the melody are sourced from the chord matrix
                            if timegap == True:
                                track.append(Message('note_off', note=cmajchords[note_status[voice],chord_progression[chord]], velocity=volume_status[voice], time=int(chord*measure_length+beat*measure_length/num_subdiv-t)))
                                t = int(chord*measure_length+beat*measure_length/num_subdiv)
                                timegap = False
                            else:
                                track.append(Message('note_off', note=cmajchords[note_status[voice],chord_progression[chord]], velocity=volume_status[voice], time=0))  
                    if voice == 'melody': 
                        volume_status[voice] = volume_control(voices[voice][chord][beat])
                        if timegap == True:
                            track.append(Message('note_on', note=cmajorscale[voices[voice][chord][beat]], velocity=volume_status[voice], time=int(chord*measure_length+beat*measure_length/num_subdiv-t)))
                            t = int(chord*measure_length+beat*measure_length/num_subdiv)
                            timegap = False
                        else:
                            track.append(Message('note_on', note=cmajorscale[voices[voice][chord][beat]], velocity=volume_status[voice], time=0))
                        note_status[voice] = voices[voice][chord][beat] 
                    else:
                        volume_status[voice] = volume_control(chord_progression[chord])-10
                        if timegap == True:
                            track.append(Message('note_on', note=cmajchords[voice_chord_index.index(voice),chord_progression[chord]], velocity=volume_status[voice], time=int(chord*measure_length+beat*measure_length/num_subdiv-t)))
                            t = int(chord*measure_length+beat*measure_length/num_subdiv)
                            timegap = False
                        else:
                            track.append(Message('note_on', note=cmajchords[voice_chord_index.index(voice),chord_progression[chord]], velocity=volume_status[voice], time=0))
                        note_status[voice] = voice_chord_index.index(voice)    
    #hold all notes until the end of the last measure and then cut them all off
    track.append(Message('note_off', note=cmajchords[0,chord_progression[chord]], velocity=volume_status['bass'], time=int(len(chord_progression)*measure_length-t)))    
    track.append(Message('note_off', note=cmajchords[1,chord_progression[chord]], velocity=volume_status['tenor'], time=0))
    track.append(Message('note_off', note=cmajchords[2,chord_progression[chord]], velocity=volume_status['alto'], time=0))
    track.append(Message('note_off', note=cmajchords[3,chord_progression[chord]], velocity=volume_status['lead'], time=0))
    track.append(Message('note_off', note=cmajorscale[note_status['melody']], velocity=volume_status['melody'], time=0))

measure_length = np.random.choice([1120,2240,4480], p= [.3,.3,.4])

num_subdiv = np.random.choice([8,6],p=[.6,.4])

print('time signature: ',num_subdiv)

num_chords = 1 + random.choice([4,8,12])

chord_progression = []
i=1
while i < num_chords: 
    chord_progression.append(random.randint(0,6))
    i=i+1

beat_values = list(range(num_subdiv))  # possible rhythmic locations in each measure
if num_subdiv == 6:
    comp_rhythm = [30, 1, 1, 1, 1, 1]  # probabalistic rhythms
    melody_rhythm = [1, 1, 1, 1, 1, 1]
else:
    comp_rhythm = [30, 1, 1, 1, 1, 1, 1, 1]  # probabalistic rhythms
    melody_rhythm = [1, 1, 1, 1, 1, 1, 1, 1]
    
print('tempo: ',measure_length)
print('chord progression: ', chord_progression)

melody_line = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=melody_rhythm, max_length=5)
print('original melody: ', melody_line)
smooth_melody = melody_smoother(melody_line)
print('smoothed melody: ',smooth_melody)
declashed_melody = melody_declasher(chord_progression,smooth_melody)
print('declashed_melody: ',declashed_melody)

bass_line = generate_random_dicts(num_dicts=num_chords, keys=beat_values, weights=comp_rhythm, max_length=5)
tenor_line = generate_random_dicts(num_dicts=num_chords, keys=beat_values, weights=comp_rhythm, max_length=5)
alto_line = generate_random_dicts(num_dicts=num_chords, keys=beat_values, weights=comp_rhythm, max_length=5)
lead_line = generate_random_dicts(num_dicts=num_chords, keys=beat_values, weights=comp_rhythm, max_length=5)

build_track(melody=declashed_melody, bass=bass_line, tenor=tenor_line,alto=alto_line, lead=lead_line)

if measure_length * len(chord_progression) < 16000:
    print('short chord progression. looping progression 3 more times')
    melody2 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=melody_rhythm, max_length=5)
    smooth_melody2 = melody_smoother(melody2)
    declashed_melody2 = melody_declasher(chord_progression,smooth_melody2)
    bass_line2 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    tenor_line2 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    alto_line2 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    lead_line2 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    print('chord progression: ', chord_progression)
    print('melody2: ',melody2)
    build_track(melody=declashed_melody2, bass=bass_line2, tenor=tenor_line2,alto=alto_line2, lead=lead_line2)
    melody3 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=melody_rhythm, max_length=5)
    smooth_melody3 = melody_smoother(melody3)
    declashed_melody3 = melody_declasher(chord_progression,smooth_melody3)
    bass_line3 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    tenor_line3 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    alto_line3 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    lead_line3 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    print('chord progression: ', chord_progression)
    print('melody3: ',melody3)
    build_track(melody=declashed_melody3, bass=bass_line3, tenor=tenor_line3,alto=alto_line3, lead=lead_line3)
    melody4 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=melody_rhythm, max_length=5)
    smooth_melody4 = melody_smoother(melody4)
    declashed_melody4 = melody_declasher(chord_progression,smooth_melody4)
    bass_line4 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    tenor_line4 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    alto_line4 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    lead_line4 = generate_random_dicts(num_dicts=num_chords-1, keys=beat_values, weights=comp_rhythm, max_length=5)
    print('chord progression: ', chord_progression)
    print('melody4: ',melody4)
    build_track(melody=declashed_melody4, bass=bass_line4, tenor=tenor_line4,alto=alto_line4, lead=lead_line4)
    
mid.save('output.mid')

time signature:  8
tempo:  4480
chord progression:  [1, 5, 1, 3, 6, 0, 6, 4, 3, 4, 6, 6]
original melody:  [{0: 5, 4: 3, 6: 3, 7: 1}, {1: 2, 2: 2, 3: 0, 5: 5, 7: 2}, {0: 6, 1: 0, 3: 0}, {2: 1, 4: 4, 5: 0}, {0: 3}, {6: 2}, {5: 1, 6: 0, 7: 3}, {2: 5}, {0: 3, 1: 6, 2: 2, 7: 5}, {4: 3}, {1: 3, 2: 0, 5: 2}, {1: 1, 2: 0, 4: 3, 5: 5}]
smoothed melody:  [{0: 5, 4: 5, 6: 4, 7: 6}, {1: 0, 2: 0, 3: 6, 5: 2, 7: 1}, {0: 6, 1: 6, 3: 6}, {2: 6, 4: 4, 5: 6}, {0: 6}, {6: 5}, {5: 5, 6: 6, 7: 6}, {2: 5}, {0: 4, 1: 3, 2: 1, 7: 1}, {4: 4}, {1: 4, 2: 5, 5: 0}, {1: 1, 2: 1, 4: 0, 5: 0}]
declashed_melody:  [{0: 5, 4: 5, 6: 4, 7: 6}, {1: 0, 2: 0, 3: 6, 5: 2, 7: 1}, {0: 6, 1: 6, 3: 6}, {2: 6, 4: 4, 5: 6}, {0: 6}, {6: 5}, {5: 5, 6: 6, 7: 6}, {2: 5}, {0: 4, 1: 3, 2: 1, 7: 1}, {4: 4}, {1: 4, 2: 5, 5: 1}, {1: 1, 2: 1, 4: 1, 5: 1}]


In [1241]:
# use fluidsynth, timidity, and pygame to play back the audio out loud
current_folder = os.getcwd()
parent_folder = os.path.dirname(current_folder)
reference_files = parent_folder+'\\reference_files'

mid.save(reference_files+'\\output.mid')
timidity_path = reference_files+'\\TiMidity++-2.15.0\\timidity.exe'
soundfont_path = reference_files+'\\Essential Keys-sforzando-v9.6.sf2'
midi_file_path = reference_files+'\\output.mid'
output_audio_path = reference_files+'\\output.wav'

subprocess.run([timidity_path, '-A', '-Ow', '-o', output_audio_path, '-T', 'wav', '-EFreverb=0', '-EFchorus=0', midi_file_path, soundfont_path])

fluidsynth_path = r'C:\ProgramData\chocolatey\bin\fluidsynth.exe' # TODO: move the file path specifications to an external file
soundfont_path = reference_files+'\\Essential Keys-sforzando-v9.6.sf2'
midi_file_path = reference_files+'\\output.mid'
output_audio_path = reference_files+'\\output.wav'

subprocess.run([fluidsynth_path, '-F', output_audio_path, soundfont_path, midi_file_path])

pygame.init()
pygame.mixer.music.load(reference_files+'\\output.mid')
pygame.mixer.music.play()